In [ ]:
!pip install --upgrade torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 144.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U transformers>=4.48.0 datasets triton lightning lightning[extra] pytorch_optimizer pandas scipy
# !pip install flash_attn==1.0.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoConfig,
)
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from huggingface_hub import upload_folder
import os
from torch.optim.lr_scheduler import LambdaLR
from pytorch_optimizer import load_optimizer

import torch._dynamo
torch._dynamo.config.suppress_errors = True

torch.set_float32_matmul_precision('high')

In [ ]:
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

# --- Configuration ---
class Config:
    DATASET_NAME = "kreasof-ai/SPL-Combined"
    MODEL_NAME = "answerdotai/ModernBERT-large"
    DIFFICULTY_LEVELS = {
        "hard": (1, 10),
        "medium": (10, 50),
        "easy": (50, 100),
    }
    BATCH_SIZE = 24
    LEARNING_RATE = 1e-5
    WEIGHT_DECAY = 1e-2
    NUM_EPOCHS = 2
    COMPILE_MODE = "max-autotune"  # Options: "default", "reduce-overhead", "max-autotune"
    USE_COMPILE = False  # Easily toggle compilation
    DYNAMIC_SHAPES = False  # Set True for variable-length inputs
    HF_USERNAME = "kreasof-ai"  # Replace with your HuggingFace username
    MODEL_REPO_ID = "kreasof-ai/SPL-large-checkpoints"  # Format: "username/model-name"
    PUSH_TO_HUB = True

cfg = Config()

# --- Enhanced Lightning Module ---
class ImpostorVerifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters()
        self.config = config
        self._init_model()
        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        self.automatic_optimization=False

    def _init_model(self):
        # Initialize core components
        self.model = AutoModel.from_pretrained(self.config.MODEL_NAME)
        self.config_hf = AutoConfig.from_pretrained(self.config.MODEL_NAME)
        self.impostor_layer = torch.nn.Linear(self.config_hf.hidden_size, 1)

        # Initial compilation if requested
        if self.config.USE_COMPILE:
            compile_kwargs = {
                "mode": self.config.COMPILE_MODE,
                "dynamic": self.config.DYNAMIC_SHAPES
            }
            self.model = torch.compile(self.model, **compile_kwargs)
            self.impostor_layer = torch.compile(self.impostor_layer, **compile_kwargs)

    def forward(self, input_ids, attention_mask):
        torch.compiler.cudagraph_mark_step_begin()
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        token_outputs = outputs.last_hidden_state
        return self.impostor_layer(token_outputs).squeeze(-1)

    def training_step(self, batch, batch_idx):
        impostor_logits = self.forward(input_ids=batch["input_ids"],
                             attention_mask=batch["attention_mask"])

        # Calculate masked loss
        loss = self.loss_fn(impostor_logits, batch["labels"].float())
        masked_loss = (loss * batch["attention_mask"]).sum() / (batch["attention_mask"]).sum()

        self.log("train_loss", masked_loss, prog_bar=True)

        opt = self.optimizers()

        opt.zero_grad()  # Zero gradients

        self.manual_backward(masked_loss)  # Backpropagation
        self.clip_gradients(opt, gradient_clip_val=1.0, gradient_clip_algorithm="norm")

        opt.step()  # Update weights

        return masked_loss

    def validation_step(self, batch, batch_idx):
        impostor_logits = self.forward(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

        # Calculate masked loss
        loss = self.loss_fn(impostor_logits, batch["labels"].float())
        masked_loss = (loss * batch["attention_mask"]).sum() / (batch["attention_mask"]).sum()

        # Log validation loss
        self.log("val_loss", masked_loss, prog_bar=True, sync_dist=True)

        return masked_loss

    def configure_optimizers(self):
        optimizer = load_optimizer(optimizer='muon')(
            self.parameters(),
            lr=self.config.LEARNING_RATE,
            weight_decay=self.config.WEIGHT_DECAY,
        )

        return optimizer

model = ImpostorVerifier(cfg)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [ ]:
!wget -O last.ckpt https://huggingface.co/kreasof-ai/SPL-Large-Experimental/resolve/main/SPL-epoch=00.ckpt

--2025-03-25 07:49:03--  https://huggingface.co/kreasof-ai/SPL-Large-Experimental/resolve/main/SPL-epoch=00.ckpt
Resolving huggingface.co (huggingface.co)... 13.35.202.121, 13.35.202.40, 13.35.202.97, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.121|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/aa/b5/aab54a21c7c07a1d5ae7776a62ac2f6cd5e5011c511dbfb7a52cbbe9c3aa899e/082f267af76573a2623a7c54ee891a8cf95f8128502b341aefbeaf2709f35411?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27SPL-epoch%25253D00.ckpt%3B+filename%3D%22SPL-epoch%253D00.ckpt%22%3B&Expires=1742892543&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0Mjg5MjU0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2FhL2I1L2FhYjU0YTIxYzdjMDdhMWQ1YWU3Nzc2YTYyYWMyZjZjZDVlNTAxMWM1MTFkYmZiN2E1MmNiYmU5YzNhYTg5OWUvMDgyZjI2N2FmNzY1NzNhMjYyM2E3YzU0ZWU4OTFhOGNmOTVmODEyODUwMmIzNDFhZWZiZWFmMjcwOW

In [ ]:
examples = [
    {
        "text": "Habibullah Akbar",
        "expected_quality": "irrelevant sequence",
    },
    {
        "text": "Cat sat on the mat",
        "expected_quality": "irrelevant sequence",
    },
    {
        "text": "The cat sat on the mat, basking in the warm sunlight streaming through the window, its tail gently flicking back and forth as it dozed off into a peaceful nap.",
        "expected_quality": "irrelevant sequence",
    },
    {
        "text": "As a professional AI language model, I don't have personal experiences or emotions, nor do I engage in hobbies or leisure activities. My purpose is to provide accurate and informative responses to assist users with their queries, and I do not possess the capacity to experience personal preferences or enjoyment. I am solely focused on delivering high-quality information and maintaining a professional tone in my interactions.",
        "expected_quality": "irrelevant sequence",
    },
    {
        "text": "To simplify the algebraic expression `(3x^2 - 4y^3) / (2x)`, we can follow a few steps: Step 1: Distribute the division symbol by multiplying the expression by the reciprocal of the denominator. The reciprocal of `2x` is `1/(2x)`, so the expression becomes `(3x^2 - 4y^3) * (1/(2x))`. Step 2: Simplify within the parentheses by dividing each term separately. - For the first term, `3x^2`, divide `3x^2` by `2x`. This gives us `(3x^2) / (2x) = (3/2) * (x^2 / x) = (3/2) * x`. - For the second term, `-4y^3`, divide `-4y^3` by `2x`. This gives us `(-4y^3) / (2x) = (-2) * (y^3 / x)`. Step 3: Combine the simplified terms from Step 2. The expression now becomes `(3/2) * x - 2 * (y^3 / x)`. So, the simplified form of the algebraic expression `(3x^2 - 4y^3) / (2x)` is `(3/2) * x - 2 * (y^3 / x)`.",
        "expected_quality": "higher score",
    },
    {
        "text": "To simplify the algebraic expression `(3x^2 - 4y^3) / (2x)`, you can divide each term in the numerator by the denominator. First, let's divide `3x^2` by `2x`. Since both terms have a common factor of `x`, we can simplify this expression to `3x`. Next, we divide `-4y^3` by `2x`. We can simplify this expression by dividing each term separately. Dividing `-4` by `2` gives `-2`. Then, dividing `y^3` by `x` gives `y^3/x`. So, the simplified form of `(3x^2 - 4y^3) / (2x)` is `3x - 2y^3/x`.",
        "expected_quality": "lower score",
    },
    {
        "text": "Proof that 1 = 2. Let’s start with two equal numbers, \( a = b \). 1. Multiply both sides by \( a \): \( a^2 = ab \). 2. Subtract \( b^2 \) from both sides: \( a^2 - b^2 = ab - b^2 \). 3. Factor both sides: \( (a - b)(a + b) = b(a - b) \). 4. Divide both sides by \( (a - b) \): \( a + b = b \). 5. Since \( a = b \), substitute \( b \) for \( a \): \( b + b = b \) → \( 2b = b \). 6. Divide both sides by \( b \): \( 2 = 1 \).",
        "expected_quality": "logical fallacy",
    },
    {
        "text": "Let’s start with two equal numbers, \( a = b \). 1. Multiply both sides by \( a \): \( a^2 = ab \). 2. Subtract \( b^2 \) from both sides: \( a^2 - b^2 = ab - b^2 \). 3. Factor both sides: \( (a - b)(a + b) = b(a - b) \). 4. Divide both sides by \( (a - b) \): \( a + b = b \). 5. Since \( a = b \), substitute \( b \) for \( a \): \( b + b = b \) → \( 2b = b \). 6. Divide both sides by \( b \): \( 2 = 1 \).",
        "expected_quality": "logical fallacy",
    },
    {
        "text": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Natalia sold 48/2 = <<48/2=24>>24 clips in May. Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May. #### 72",
        "expected_quality": "right answer",
    },
    {
        "text": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50. Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30. This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more. #### 5",
        "expected_quality": "wrong answer",
    }
]

In [ ]:
model = ImpostorVerifier.load_from_checkpoint('last.ckpt')
tokenizer = AutoTokenizer.from_pretrained(cfg.MODEL_NAME)

def min_max_scale(scores, min_score=-1, max_score=0):
    return (scores - min_score) / (max_score - min_score)

model.eval()
model.model.eval()

# model.model = model.model._orig_mod  # Access the original model
# model.impostor_layer = model.impostor_layer._orig_mod

for example in examples:
    text = example["text"]
    expected_quality = example["expected_quality"]

    inputs = tokenizer(
        text,
        padding="max_length",
        max_length=2048,
        truncation=True,
        return_tensors="pt"
    ).to('cuda')

    output_logits = model(**inputs)
    output_probs = torch.sigmoid(output_logits)

    sequence_quality_raw = -torch.sum(torch.log(1 - output_probs + 1e-12), dim=1).item() # Negative Log-Likelihood
    sequence_quality_revised = min_max_scale(-sequence_quality_raw) # Min-Max Scaling

    # --- Print Results ---
    print(f"Text: {text}")
    print(f"Expected Quality: {expected_quality}")
    print(f"Token Probabilities: {output_probs.squeeze(0).tolist()}")
    print(f"Sequence Quality (Negative log-likelihood): {sequence_quality_raw}")
    print(f"Sequence Quality Revised: {sequence_quality_revised}")

    print("-" * 20)

Text: Habibullah Akbar
Expected Quality: irrelevant sequence
Token Probabilities: [0.0156823992729187, 0.02040841057896614, 0.016342058777809143, 0.028894927352666855, 0.01951068639755249, 0.006373525597155094, 0.017775725573301315, 0.056302737444639206, 0.057177361100912094, 0.05432719737291336, 0.02953527309000492, 0.03135054185986519, 0.023273101076483727, 0.013010634109377861, 0.009068649262189865, 0.004308873787522316, 0.008383698761463165, 0.015390321612358093, 0.016426414251327515, 0.011297236196696758, 0.016614658758044243, 0.019035881385207176, 0.01188649795949459, 0.013881796039640903, 0.014908849261701107, 0.012163110077381134, 0.0071205319836735725, 0.024641506373882294, 0.025637594982981682, 0.02371343784034252, 0.016375388950109482, 0.01129695400595665, 0.0075216395780444145, 0.047729674726724625, 0.05016966909170151, 0.0045163072645664215, 0.016227344051003456, 0.016221284866333008, 0.010252800770103931, 0.006468090694397688, 0.003897744696587324, 0.006702304817736149, 0

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from datasets import load_dataset

def get_score(problem, answer):
    """Replace with your actual model inference code"""
    inputs = tokenizer(
        problem + " " + answer,
        padding="max_length",
        max_length=2048,
        truncation=True,
        return_tensors="pt"
    ).to('cuda')

    output_logits = model(**inputs)
    output_probs = torch.sigmoid(output_logits)

    sequence_quality_raw = -torch.sum(torch.log(1 - output_probs + 1e-12), dim=1).item() # Negative Log-Likelihood
    sequence_quality_revised = min_max_scale(-sequence_quality_raw) # Min-Max Scaling

    return sequence_quality_revised

# Load dataset
dataset = load_dataset("kreasof-ai/MATH-WD-Lite")
df = dataset['train'].to_pandas()

# Initialize metrics storage
results = {
    'correct_highest': [],
    'delta_max_wrong': [],
    'delta_avg_wrong': [],
    'correct_length': [],
    'decoy_lengths': [],
    'levels': [],
    'level_accuracies': {l: [] for l in range(1, 6)},
    'level_deltas': {l: [] for l in range(1, 6)}
}

# Process each question
for _, row in df.iterrows():
    problem = row['Problem']
    candidates = {
        'correct': row['Answer'],
        'decoy_a': row['Decoy A'],
        'decoy_b': row['Decoy B'],
        'decoy_c': row['Decoy C']
    }

    # Get scores and lengths
    scores, lengths = {}, {}
    for key, answer in candidates.items():
        scores[key] = get_score(problem, answer)
        lengths[key] = len(answer.split())  # Simple length approximation

    # Store results
    decoy_scores = [scores['decoy_a'], scores['decoy_b'], scores['decoy_c']]
    results['correct_highest'].append(scores['correct'] > max(decoy_scores))
    results['delta_max_wrong'].append(scores['correct'] - max(decoy_scores))
    results['delta_avg_wrong'].append(scores['correct'] - np.mean(decoy_scores))
    results['correct_length'].append(lengths['correct'])
    results['decoy_lengths'].extend([lengths[k] for k in ['decoy_a', 'decoy_b', 'decoy_c']])
    results['levels'].append(row['Level'])

    # Track level-based metrics
    results['level_accuracies'][row['Level']].append(results['correct_highest'][-1])
    results['level_deltas'][row['Level']].append(results['delta_max_wrong'][-1])

# Calculate metrics
# 1. Accuracy
accuracy = np.mean(results['correct_highest'])

# 2. Score gaps
mean_delta_max = np.mean(results['delta_max_wrong'])
mean_delta_avg = np.mean(results['delta_avg_wrong'])

# 3. Sequence length analysis
all_lengths = results['correct_length'] + results['decoy_lengths']
all_scores = [scores['correct']] + decoy_scores  # Changed logic here
                                                  # to include scores for all answers

# Now, all_scores will contain:
# - the score for the correct answer
# - the scores for decoy_a, decoy_b, decoy_c

# After this change, you'll need to extend all_scores for each question, similar to how you extended all_lengths:
all_scores = []
for i in range(len(results['correct_highest'])):
    all_scores.extend([results['delta_max_wrong'][i] + scores['correct'] if results['correct_highest'][i] else scores['correct'],
                       scores['decoy_a'],
                       scores['decoy_b'],
                       scores['decoy_c']])

length_corr, _ = pearsonr(all_lengths, all_scores)

# 4. Level correlation
level_accs = [np.mean(results['level_accuracies'][l]) for l in range(1, 6)]
level_deltas = [np.mean(results['level_deltas'][l]) for l in range(1, 6)]
level_numbers = list(range(1, 6))
level_acc_corr, _ = spearmanr(level_numbers, level_accs)
level_delta_corr, _ = spearmanr(level_numbers, level_deltas)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Mean Δ(max wrong): {mean_delta_max:.4f}")
print(f"Mean Δ(avg wrong): {mean_delta_avg:.4f}")
print(f"\nLength-Score Correlation: {length_corr:.4f}")
print(f"\nLevel vs Accuracy Correlation: {level_acc_corr:.4f}")
print(f"Level vs Δ(max wrong) Correlation: {level_delta_corr:.4f}")

# Additional analysis
print("\nAccuracy by Level:")
for l in range(1, 6):
    print(f"Level {l}: {np.mean(results['level_accuracies'][l]):.4f}")

print("\nΔ(max wrong) by Level:")
for l in range(1, 6):
    print(f"Level {l}: {np.mean(results['level_deltas'][l]):.4f}")

Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Accuracy: 0.3438
Mean Δ(max wrong): -0.1281
Mean Δ(avg wrong): -0.0148

Length-Score Correlation: -0.0116

Level vs Accuracy Correlation: -0.7000
Level vs Δ(max wrong) Correlation: -0.3000

Accuracy by Level:
Level 1: 0.4000
Level 2: 0.3514
Level 3: 0.3871
Level 4: 0.3684
Level 5: 0.2564

Δ(max wrong) by Level:
Level 1: -0.0424
Level 2: -0.1701
Level 3: -0.1028
Level 4: -0.1580
Level 5: -0.1123
